In [1]:
import os
import sys
import pathlib
import platform

import numpy as np
import nibabel as nib

from typing import (
    Dict,
    List,
    Optional,
    Tuple
)

In [2]:
os.getcwd()

'/Users/adebayobraimah/Desktop/projects/convert_source'

In [3]:
from convert_source.cs_utils.fileio import (
    Command,
    DependencyError,
    File
)

In [20]:
from convert_source.cs_utils.utils import (
    SubDataInfo,
    collect_info,
    list_in_substr,
    write_json
)

In [15]:
from convert_source.imgio.niio import (
    get_num_frames,
    get_nii_tr,
    get_num_frames
)

In [12]:
data_dir: str = "/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir"
nii_test_data: str = os.path.join(data_dir,'TEST001-UNIT001','data.nifti')

# Code to create empty test NIFTI-2 files
nii_list: List[str] = ['T1_AXIAL','T2_InPlane','DWI_68_DIR','rs_fMRI','rsfMRI','DWI_B0','FLAIR']


In [32]:
def create_nifti_image(name: str,
                       num_frames: Optional[int] = 1,
                       tr: Optional[float] = 2.00,
                       task: Optional[str] = ""
                       ) -> Tuple[str,str]:
    '''Creates an empty NIFTI-2 image using the specified repetition time (TR, in sec.), and number of frames (/TRs).
    A corresponding JSON sidecar is also created for the NIFTI-2 file.

    Usage example:
        >>> [nii,json] = create_nifti_image("test.nii.gz")

    Arguments:
        name: Output NIFTI-2 filename.
        num_frames: Number of frames/TRs for the file to have.
        tr: Repetition time (s)
        task: Task name (e.g. "Resting state", "N-back", etc.)

    Returns:
        Tuple of strings that represent:
            * File path to NIFTI-2 image as a string.
            * File path to corresponding JSON sidecar.
    '''
    # Create empty NIFTI-2 file
    data = np.arange(4*4*3).reshape(4,4,3)
    new_image = nib.Nifti2Image(data, affine=np.eye(4))
    img_header = new_image.header

    img_header['dim'][3] = num_frames
    img_header['pixdim'][4] = tr

    nib.save(new_image, name)

    # Write JSON
    params: Dict = {"RepetitionTime":tr}

    if task:
        tmp_dict: Dict = {"TaskName":task}
        params.update(tmp_dict)
    
    # File class context manager to get filename for JSON file
    with File(name) as f:
        [path, filename, ext] = f.file_parts()
        json_name: str = os.path.join(path,filename + ".json")
    
    json_name: str = write_json(json_file=json_name,
                                dictionary=params)

    return (name,
            json_name)

def create_test_files(test_gzip: bool = False):
    if test_gzip:
        ext: str = '.nii.gz'
    else:
        ext: str = '.nii'

    for nii in nii_list:
        if list_in_substr(['dwi'],nii):
            num_frames: int = 68
            tr: float = 1.2
            task: str = ""
        elif list_in_substr(['rsfMRI'],nii):
            num_frames: int = 68
            tr: float = 2.00
            task: str = "Resting State"
        else:
            num_frames: int = 1
            tr: float = 0
            task: str = ""
        
        nii_file: str = os.path.join(nii_test_data,nii + ".nii")

        print(num_frames)
        print(tr)
        print(task)

        create_nifti_image(name=nii_file,
                           num_frames=num_frames,
                           tr=tr,
                           task=task)

        if 'dwi' in nii.lower():
            with File(nii_file) as f:
                [path,filename,ext] = f.file_parts()

                bval: str = os.path.join(path,filename + ".bval")
                bvec: str = os.path.join(path,filename + ".bvec")

                with File(bval) as b:
                    b.touch
                    b.write_txt("0 0 0 0 0 800 1000 1500 2000 3000")
                    
                with File(bvec) as e:
                    e. touch()

In [38]:
create_test_files(test_gzip=False)

1
0

1
0

68
1.2

1
0

68
2.0
Resting State
68
1.2

1
0



In [39]:
subs_data: List[SubDataInfo] = collect_info(parent_dir=data_dir,
                                                exclusion_list=[".dcm",".PAR"])

In [40]:
subs_data

[{'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_68_DIR.nii'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_B0.nii'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/FLAIR.nii'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/T1_AXIAL.nii'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/T2_InPlane.nii'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/

In [42]:
get_nii_tr(subs_data[0].data)

1.0

In [45]:
get_num_frames(subs_data[5].data)

1

In [28]:
'dwi' in nii_list[2].lower()

True

In [29]:
list_in_substr(['dwi'],nii_list[2])

True

In [26]:
nii_list

['T1_AXIAL',
 'T2_InPlane',
 'DWI_68_DIR',
 'rs_fMRI',
 'rsfMRI',
 'DWI_B0',
 'FLAIR']

In [46]:
def create_nifti_image(name: str,
                       num_frames: Optional[int] = 1,
                       tr: Optional[float] = 2.00,
                       task: Optional[str] = ""
                       ) -> Tuple[str,str]:
    '''Creates an empty NIFTI-2 image using the specified repetition time (TR, in sec.), and number of frames (/TRs).
    A corresponding JSON sidecar is also created for the NIFTI-2 file.

    Usage example:
        >>> [nii,json] = create_nifti_image("test.nii.gz")

    Arguments:
        name: Output NIFTI-2 filename.
        num_frames: Number of frames/TRs for the file to have.
        tr: Repetition time (s)
        task: Task name (e.g. "Resting state", "N-back", etc.)

    Returns:
        Tuple of strings that represent:
            * File path to NIFTI-2 image as a string.
            * File path to corresponding JSON sidecar.
    '''
    # Create empty NIFTI-2 file
    data = np.arange(4*4*3).reshape(4,4,3)
    new_image = nib.Nifti2Image(data, affine=np.eye(4))
    img_header = new_image.header

    img_header['dim'][3] = num_frames
    img_header['pixdim'][4] = tr

    nib.save(new_image, name)

    # Write JSON
    params: Dict = {"RepetitionTime":tr}

    if task:
        tmp_dict: Dict = {"TaskName":task}
        params.update(tmp_dict)
    
    # File class context manager to get filename for JSON file
    with File(name) as f:
        [path, filename, ext] = f.file_parts()
        json_name: str = os.path.join(path,filename + ".json")
    
    json_name: str = write_json(json_file=json_name,
                                dictionary=params)

    return (name,
            json_name)

In [47]:
tt = "/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.nii"

In [48]:
create_nifti_image(tt,500,2.0,"Resting State")

('/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.nii',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.json')

In [49]:
get_nii_tr(tt)

1.0

In [50]:
get_num_frames(tt)

1

In [51]:
# Create empty NIFTI-2 file
data = np.arange(4*4*3).reshape(4,4,3)
new_image = nib.Nifti2Image(data, affine=np.eye(4))
img_header = new_image.header

In [52]:
num_frames = 500
tr = 2.0

In [60]:
img_header['dim'][3] = num_frames
img_header['pixdim'][4] = tr

In [54]:
img_header['dim']

array([  3,   4,   4, 500,   1,   1,   1,   1])

In [55]:
nib.save(new_image, tt)

In [56]:
get_nii_tr(tt)

1.0

In [65]:
img_header.set_dim_info(1,2,0)

In [61]:
print(img_header)

<class 'nibabel.nifti2.Nifti2Header'> object, endian='<'
sizeof_hdr      : 540
magic           : b'n+2'
eol_check       : [13 10 26 10]
datatype        : int64
bitpix          : 64
dim             : [  3   4   4 500   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
pixdim          : [1. 1. 1. 1. 2. 1. 1. 1.]
vox_offset      : 0
scl_slope       : nan
scl_inter       : nan
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
slice_start     : 0
slice_end       : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 0.0
qoffset_y       : 0.0
qoffset_z       : 0.0
srow_x          : [1. 0. 0. 0.]
srow_y          : [0. 1. 0. 0.]
srow_z          : [0. 0. 1. 0.]
slice_code      : unknown
xyzt_units      : 0
intent_code     : none
intent_name     : b''
dim_info        : 0
unused_str      : b''


In [66]:
print(img_header)

<class 'nibabel.nifti2.Nifti2Header'> object, endian='<'
sizeof_hdr      : 540
magic           : b'n+2'
eol_check       : [13 10 26 10]
datatype        : int64
bitpix          : 64
dim             : [  3   4   4 500   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
pixdim          : [1. 1. 1. 1. 2. 1. 1. 1.]
vox_offset      : 0
scl_slope       : nan
scl_inter       : nan
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
slice_start     : 0
slice_end       : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 0.0
qoffset_y       : 0.0
qoffset_z       : 0.0
srow_x          : [1. 0. 0. 0.]
srow_y          : [0. 1. 0. 0.]
srow_z          : [0. 0. 1. 0.]
slice_code      : unknown
xyzt_units      : 0
intent_code     : none
intent_name     : b''
dim_info        : 30
unused_str      : b''
